# New script to create (transient) storm surge heights
(This needs to be combined with sea level rise scenario to obtain storm surge *levels*

In [1]:
import sys

sys.path.append("..") #import folder which is one level higher

from classes import *
import numpy as np
import pandas as pd

### Gumbel equation
# $ F(x,\mu,\beta) = exp(-e^{(\mu-x)/\beta}) $

### Generate new SurgeHeight series using the Gumbel distribution for Hoek van Holland

Sterl, A., van den Brink, H., de Vries, H., Haarsma, R., & van Meijgaard, E. (2009). An ensemble study of extreme storm surge related water levels in the North Sea in a changing climate. Ocean Science, 5(3), 369–378. https://doi.org/10.5194/os-5-369-2009

In [5]:
mu = 2.33
beta = 0.234

In [13]:
sh1 = SurgeHeight('example_surgeheight')
sh1.from_Gumbel(2020,2300,mu=mu,beta=beta,csv_path = sh1.name + '.csv')
#allSurgeHeight #attributed to this 'global' list

### Or load them from earlier create csv files

In [2]:
#allSurgeHeight = []
# Load all SurgeHeight (Realisations of extreme value distribution)
folder = "Old_transients"
for filename in os.listdir(folder):
    if filename.endswith(".csv"):
        obj = SurgeHeight(filename.split('.')[0]) #Init object with name derived from filename
        obj.from_csv(os.path.join(folder,filename)) #Fill the object with data from the csv file

In [7]:
for SH in allSurgeHeight:
    print(SH.name ,end=' - ')

0 - 1 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 

In [10]:
SH.__dict__.keys()

dict_keys(['name', 'years', 'surgeheight'])

### Now, let's study some statistics of these good old transients

In [15]:
#What are the maximum return periods in these series?
mu = 2.33
beta = 0.234

for SH in allSurgeHeight:
    RPs = []
    for h in SH.surgeheight: #iterate over all the water levels
        RPs.append(Gumbel_RP(h,mu,beta))
    print(SH.name, int(round(max(RPs),0)))

0 359
1 1221
10 149
11 1886
12 144
13 186
14 439
15 665
16 961
17 170
18 154
19 1668
2 174
3 255
4 772
5 1557
6 164
7 416
8 82
9 216


So, indeed, these were not very extreme return periods.

# New approach: make a very, very long time series first

In [24]:
#This will almost always contain a 1:100_000 year event
new_list = np.random.gumbel(mu,beta,size=100_000)
RPs = []
for h in new_list: #iterate over all the water levels
    RPs.append(Gumbel_RP(h,mu,beta))
print(int(round(max(RPs),0)))

128302


We want to make a large number of transient series, which at least cover some 1:10,000 year events
How much transients should we make, given that the lengt of our series in fact is only 121 years (2020-2200) (and 100 year extra, but this is not further used)

In [36]:
n = 20
total_size = 20*121
print('total size = ', total_size)
new_list  = np.random.gumbel(mu,beta,size=total_size)
RPs = []
for h in new_list: #iterate over all the water levels
    RPs.append(Gumbel_RP(h,mu,beta))
print(int(round(max(RPs),0)))

total size =  2420
2326


This size gives a serious probability that the 1:10,000 year event indeed is not in. Say we want to have it at least 5 times in, we should have:

In [37]:
5*10_000/121

413.22314049586777

So that is order hundreds of transients!

In [68]:
n = 500
total_size = n*121
print('total size = ', total_size)
new_list  = np.random.gumbel(mu,beta,size=total_size)
RPs = []
over10_000 = []

for h in new_list: #iterate over all the water levels
    RP = Gumbel_RP(h,mu,beta)
    RPs.append(RP)
    if RP >= 10_000:
        over10_000.append(RP)
print(int(round(max(RPs),0)),len(over10_000))

total size =  60500
22961 8


Rule of thumb: if you want 5 events with RP>10,000, you need ~ 5x100 = 500 transients

### Can't that just be done with the old function? Let's try it.

In [6]:
#Create five hundred series
for i in range(0,501):
    name = 'five_hundred_' + str(i) 
    folder = 'five_hundred'
    
    sh = SurgeHeight('name')
    sh.from_Gumbel(2020,2300,mu=mu,beta=beta,csv_path = os.path.join(folder,name + '.csv'))

### Reload them to study their characteristics

In [2]:
#What are the maximum return periods in these series?
mu = 2.33
beta = 0.234

# Load all SurgeHeight (Realisations of extreme value distribution)
folder = "five_hundred"
for filename in os.listdir(folder):
    if filename.endswith(".csv"):
        obj = SurgeHeight(filename.split('.')[0]) #Init object with name derived from filename
        obj.from_csv(os.path.join(folder,filename)) #Fill the object with data from the csv file

In [3]:
#What are the maximum return periods in these series?
mu = 2.33
beta = 0.234

bingos = 0

for SH in allSurgeHeight:
    RPs = []
    for h in SH.surgeheight[0:121]: #iterate over all the water levels, but only the first 121 years
        RPs.append(Gumbel_RP(h,mu,beta))
    print(SH.name, int(round(max(RPs),0)))
    if max(RPs) >= 10_000:
        print('\n BINGO \n')
        bingos += 1
print(bingos)

five_hundred_0 779
five_hundred_1 433
five_hundred_10 221
five_hundred_100 101
five_hundred_101 38
five_hundred_102 152
five_hundred_103 53
five_hundred_104 564
five_hundred_105 157
five_hundred_106 79
five_hundred_107 498
five_hundred_108 81
five_hundred_109 155
five_hundred_11 96
five_hundred_110 909
five_hundred_111 275
five_hundred_112 567
five_hundred_113 28
five_hundred_114 175
five_hundred_115 119
five_hundred_116 574
five_hundred_117 85
five_hundred_118 3024
five_hundred_119 3730
five_hundred_12 464
five_hundred_120 37
five_hundred_121 471
five_hundred_122 420
five_hundred_123 728
five_hundred_124 13190

 BINGO 

five_hundred_125 149
five_hundred_126 1013
five_hundred_127 49
five_hundred_128 50
five_hundred_129 273
five_hundred_13 29
five_hundred_130 125
five_hundred_131 160
five_hundred_132 99
five_hundred_133 7851
five_hundred_134 195
five_hundred_135 553
five_hundred_136 70
five_hundred_137 68
five_hundred_138 100
five_hundred_139 522
five_hundred_14 43
five_hundred_140 456
